In [ ]:
from notebook_environment import *

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
with gzip.open('../data/ccle.pickle.gz') as f:
    CCLE = pickle.load(f)

In [ ]:
kras_mutants = CCLE['Mutation']['df'].columns[CCLE['Mutation']['df'].loc[
    'KRAS_MUT'].astype(bool)]

h_matrix = pd.read_table('../results/nmfccs/nmf_k9_h.txt', index_col=0)

h_matrix = h_matrix[h_matrix.columns & kras_mutants]

h_matrix.to_csv('../results/kras_mutant_h_matrix.txt', sep='\t')

h_matrix

In [ ]:
ccal.plot_heatmap(
    h_matrix,
    normalization_method='-0-',
    normalization_axis=1,
    cluster=True,
    xlabel='Cellline',
    ylabel='KRAS Component',
    xticklabels=False)

In [ ]:
ccal.make_comparison_panel(
    h_matrix,
    h_matrix,
    axis=1,
    array_2d_0_name='Component',
    array_2d_1_name='Component')

In [ ]:
h_matrix = h_matrix.loc[['C7', 'C2', 'C1']]

In [ ]:
# '-0-' normalize each row
a = ccal.normalize_2d_array(h_matrix.values, '-0-', axis=1)

# Clip values 3 standard deviation away from each row
a = a.clip(min=-3, max=3)

# '0-1' normalize each row
a = ccal.normalize_2d_array(a, '0-1', axis=1)

h_matrix = pd.DataFrame(a, index=h_matrix.index, columns=h_matrix.columns)

# ds, hcs, hccs, cccs = ccal.define_states(
#     h_matrix,
#     range(2, 8),
#     '../results/hccs',
#     n_clusterings=50,
#     random_seed=12765)

In [ ]:
hccs = pd.read_table('../results/hccs/hccs.txt', index_col=0)

sample_states = hccs.loc['K4']

sample_states.name = 'State'

sample_states.index.name = 'Cellline'
sample_states.name = 'KRAS Mutant Sample State'

sample_states.to_csv(
    '../results/kras_mutant_sample_states.txt', sep='\t', header=True)

# Rename states to match the paper
sample_states = sample_states.map({2: 0, 3: 1, 0: 2, 1: 3})

In [ ]:
PURPLE = '#807DBA'
BLUE = '#4292C6'
GREEN = '#41AB5D'
RED = '#EF3B2C'

ccal.plot_heatmap(
    h_matrix,
    data_type='continuous',
    normalization_method='-0-',
    normalization_axis=1,
    column_annotation=sample_states,
    annotation_colors=[PURPLE, BLUE, RED, GREEN],
    title='Clustered KRAS H-Matrix',
    xlabel='Sample',
    ylabel='Component',
    xticklabels=False)

In [ ]:
gps_map = ccal.GPSMap(
    h_matrix,
    pull_power=1.2,
    node_x_dimension=pd.DataFrame(
        [[0.5, np.sqrt(3) / 2], [1, 0], [0, 0]], index=h_matrix.index))

gps_map.set_sample_phenotypes(
    sample_states,
    phenotype_type='categorical',
    bandwidth_factor=2.8,
    phenotype_color_map=mpl.colors.ListedColormap([PURPLE, BLUE, RED, GREEN]),
    phenotype_to_str={i: 'State {}'.format(i + 1)
                      for i in range(4)})

gps_map.plot_samples_with_phenotype()

gps_map.save('../results/kras_mutant_gps_map.pickle.gz')